<a href="https://colab.research.google.com/github/jjschueder/7333QTW/blob/master/Case%20Study%203/Case_Study_3_Gaussian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CASE 3/CASE 6

### Summary/Introduction blurb

### Data Processing 

We received the data in an "R data format" file. Since we opted to do this analysis in Python, the Python package "pyreadr" was used to read the data into a Python Pandas dataframe; we then converted the dataframe to a comma separated file (CSV) to store on GitHub for ease of use. Each execution of this Jupyter notebook reads the data from the CSV file on GitHub. 

https://github.com/jjschueder/7333QTW/blob/master/Case%20Study%203/Py%20read%20R.JPG -- DO WE NEED THIS LINK? I dont think so as it refers to an image file with a *.jpg extension.  Perhaps Joe can clarify.

In [0]:
#import all packages needed
import pandas as pd
# Install a conda package in the current Jupyter kernel
import sys
!pip install pyreadr
import pyreadr
from pandas_profiling import ProfileReport

     |████████████████████████████████| 266kB 2.8MB/s 


In [0]:
# import the dataset - ONLY DONE ONCE, included for reference
# result = pyreadr.read_r(r'C:\Users\jjsch\Downloads\Week_5_Materials_2\data.Rda')

In [0]:
# convert to a dataframe - ONLY DONE ONCE, for reference
# df = result["emailDFrp"]

In [0]:
# export dataframe to CSV - ONLY DONE ONCE, for reference
# df.to_csv(r'C:\Users\jjsch\Downloads\Week_5_Materials_2\data.csv')

In [0]:
# view data 
df = pd.read_csv(r'https://raw.githubusercontent.com/jjschueder/7333QTW/master/Case%20Study%203/data.csv')
df.head()

,Unnamed: 0,isSpam,isRe,underscore,priority,isInReplyTo,sortedRec,subPunc,multipartText,hasImages,isPGPsigned,subSpamWords,noHost,numEnd,isYelling,isOrigMsg,isDear,isWrote,numLines,bodyCharCt,subExcCt,subQuesCt,numAtt,numRec,perCaps,hour,perHTML,subBlanks,forwards,avgWordLen,numDlr
0,0,0,T,F,F,T,T,F,F,F,F,F,F,F,F,F,F,F,50,1554,0.0,0.0,0.0,2.0,4.451039,11.0,0.0,12.500000,0.000000,4.376623,3
1,1,0,F,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,26,873,0.0,0.0,0.0,1.0,7.491289,11.0,0.0,8.000000,0.000000,4.555556,0
2,2,0,F,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,38,1713,0.0,0.0,0.0,1.0,7.436096,12.0,0.0,8.000000,0.000000,4.817164,0
3,3,0,F,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,32,1095,0.0,0.0,0.0,0.0,5.090909,13.0,0.0,18.918919,3.125000,4.714286,0
4,4,0,T,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,31,1021,0.0,0.0,0.0,1.0,6.116643,13.0,0.0,15.217391,6.451613,4.234940,0


Our initial review of the data shows many columns are formated as a "logical" data type of TRUE or FALSE. Machine learning classificaton algorithms need numeric variables. Therefore, we converted the TRUE/FALSE variables to a numeric representation: TRUE == 1; FALSE == 0. The remaining variables are numeric as they represent percentages, counts, and hours of the day.

In [0]:
# Transform logical data to numeric data
# df['isSpam'] = df['isSpam'].map({'T': 1, 'F': 0})
df['isRe'] = df['isRe'].map({'T': 1, 'F': 0})             
df['underscore'] = df['underscore'].map({'T': 1, 'F': 0})       
df['priority'] = df['priority'].map({'T': 1, 'F': 0})         
df['isInReplyTo'] = df['isInReplyTo'].map({'T': 1, 'F': 0})      
df['sortedRec'] = df['sortedRec'].map({'T': 1, 'F': 0})        
df['subPunc'] = df['subPunc'].map({'T': 1, 'F': 0})          
df['multipartText'] = df['multipartText'].map({'T': 1, 'F': 0})    
df['hasImages'] = df['hasImages'].map({'T': 1, 'F': 0})        
df['isPGPsigned'] = df['isPGPsigned'].map({'T': 1, 'F': 0})       
df['subSpamWords'] = df['subSpamWords'].map({'T': 1, 'F': 0})      
df['noHost'] = df['noHost'].map({'T': 1, 'F': 0})            
df['numEnd'] = df['numEnd'].map({'T': 1, 'F': 0})            
df['isYelling'] = df['isYelling'].map({'T': 1, 'F': 0})         
df['isOrigMsg'] = df['isOrigMsg'].map({'T': 1, 'F': 0})         
df['isDear'] = df['isDear'].map({'T': 1, 'F': 0})            
df['isWrote'] = df['isWrote'].map({'T': 1, 'F': 0})    

### Exploratory Data Analysis 

In [0]:
type(df)

pandas.core.frame.DataFrame

After confirming proper the proper type of data frame, we conducted exploratory data analysis to better understand the dataset. Our first step was to understand the variables and their associated meaning in relation to an email message. The data dictionary below provides a summary. Note, the target variable to predict - IsSpam - is included. There is a record id - Unnamed: 0 - which is not necessary for our analysi,s but we retained it for reference as needed.

#### Data Dictionary

In [0]:
df.columns

Index(['Unnamed: 0', 'isSpam', 'isRe', 'underscore', 'priority', 'isInReplyTo',
       'sortedRec', 'subPunc', 'multipartText', 'hasImages', 'isPGPsigned',
       'subSpamWords', 'noHost', 'numEnd', 'isYelling', 'isOrigMsg', 'isDear',
       'isWrote', 'numLines', 'bodyCharCt', 'subExcCt', 'subQuesCt', 'numAtt',
       'numRec', 'perCaps', 'hour', 'perHTML', 'subBlanks', 'forwards',
       'avgWordLen', 'numDlr'],
      dtype='object')

| VARIABLE | DEFINITION |
|-|-|
| isSpam | TRUE if email classified as spam |
| isRe | TRUE if Re: appears at the start of subject line |
| numLines | number of lines in body of message |
| bodyCharCt | number of characters in the body of message |
| underscore | TRUE if email address in From field of header contains underscore |
| subExcCt | number of exclamation marks in subject |
| subQuesCt | number of question marks in subject |
| numAtt | number of attachments in message |
| priority | TRUE if Priority key is present in header |
| numRec | number of recipients of message, including CCs |
| perCaps | percentage of capitals among all letters in message |
| isInReplyTo | TRUE if the In-Reply-To key is present in header |
| sortedRec | TRUE if recipients' email addresses are sorted |
| subPunc | TRUE if words in subject have punctuation or numbers embedded (i.e. w!se) |
| hour | hour of the day in the Date field |
| multipartText | TRUE if MIME type is multipart/text |
| hasImages | TRUE if message contains images |
| isPGPsigned | TRUE if message contains a PGP (encryption) signature |
| perMTML | percentage of characters in HTML tags in message compared to all characters |
| subSpamWords | TRUE if subject contains one of the words in spam word vector |
| subBlanks | percentage of blanks in subject |
| noHost | TRUE if there is no hostname in Message-Id key in header |
| numEnd | TRUE if sender's email address (before @) ends in number |
| isYelling | TRUE if subject is in all capital letters |
| forwards | number of forward symbols in a line of the body (i.e. >>> xxx contains 3) |
| isOrigMsg | TRUE if message body contains phrase "original message" |
| isDear | TRUE if message body contains word "dear" |
| isWrote | TRUE if message contains phrase "wrote:" |
| avgWordLen | average length of words in message |
| numDlr | number of dollar signs in message |

#### Data profiling

Next, we dug deeper into the exploratory analysis to understand descriptive statistics, data distrutions, and corrleation between our variables. Using standard Python command and a Python package called Pandas Profiling, we found some notable characters about select variables.

Missing data: 
* subSpam (7 missing), noHost (1 missing), isYelling (7 missing), subExcCt (20 missing), subQuesCt (20 missing), numRec (20 missing), and subBlanks (20 missing). 
* Missing values and imputation strategy outlined below.

Outliers / Odd data:
* Outliers correlated with Spam: numRec with 311 receipents; 100% capital letters in email (>80% was spam threshold for perCaps); 100% HTML characters (>97% was spam threshold for perHTML); 86% blanks in subject (>25% was spam threshold for subBlanks), and average word length of 26 is a spam outlier.
* Outliers not necessarily correlated with Spam: count of question marks in subject (8 and 12 outliers were not spam); greatest number of attachments at 18 (not spam); number of forwards at 99 (not spam); and large number of dollar signs at 1977 (not spam).

WHAT OTHER INTERESTING THINGS TO NOTE FROM PROFILE
* Distributions
* Other thoughts?


#### Missing Values and Imputation Strategy
As noted above, we addressed missing values in our dataset to improve our results. We opted to drop the 20 rows missing in the “subExcCt”, “subQuesCt”, and “subBlanks” columns. The missing data associated with these three columns was absent in the same rows; therefore, only 20 records were removed in addressing those columns. We felt this was an appropriate step given it represented only 0.2% of the data. Dropping these records also removed the seven missing values of “isYelling” and “subSpamWords.”

The number of recipients (numRec) has the most missing values at 282. We decided to impute the missing values with the mean number of recipients. This imputation was done after we split the data into training and testing datasets. The mean was calculated from the training dataset only. We felt this would most closely mimic a production environment and limit data leakage between the training and test sets. The remaining missing data point was one value for “noHost"; we imputed this with the most frequently occurring value (mode) of False (0). **{IS THIS WHAT WE DID!??}{ OR WE CAN ASSIGN THE VALUE SINCE WE KNOW THE MISSING ONE IS SPAM ... I think all the SPAMs have a noHost of TRUE}

In [0]:
# Missing Data Summary
countOfNan = pd.Series(df.isnull().sum()) 
DataType = pd.Series(df.dtypes) 
  
frame = { 'datatype': DataType, 'count of Nan': countOfNan } 
result = pd.DataFrame(frame) 
print(result)

# remove NaNs
dfNoNa = df[~df['subQuesCt'].isnull()]

              datatype  count of Nan
Unnamed: 0       int64             0
isSpam           int64             0
isRe             int64             0
underscore       int64             0
priority         int64             0
isInReplyTo      int64             0
sortedRec        int64             0
subPunc          int64             0
multipartText    int64             0
hasImages        int64             0
isPGPsigned      int64             0
subSpamWords   float64             7
noHost         float64             1
numEnd           int64             0
isYelling      float64             7
isOrigMsg        int64             0
isDear           int64             0
isWrote          int64             0
numLines         int64             0
bodyCharCt       int64             0
subExcCt       float64            20
subQuesCt      float64            20
numAtt         float64             0
numRec         float64           282
perCaps        float64             0
hour           float64             0
p

### Explanation of Data Split, equivalent to R split()

In the Python sklearn package, "Stratified Shuffle Split" provides train/test indices to split data in train/test sets.
This cross-validation object is a merge of StratifiedKFold and ShuffleSplit, which returns stratified randomized folds. The folds are made by preserving the percentage of samples for each class. https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html  

The settings we used for training are 1 fold using eighty percent of the data for training. Twenty percent of the data will be used to test the initial model. 

** DID WE USE STRATIFIED SHUFFLE SPLIT OR JUST SHUFFLE SPLIT ?? It looks like we commented out Strat Shuffle Split. Why did we decide to do that? Why was Shuffle Split better than Strat Shuffle Split??

In [0]:
# now divide the data into test and train using scikit learn built-ins
from sklearn.model_selection import StratifiedShuffleSplit 
from sklearn.model_selection import ShuffleSplit
cvx = ShuffleSplit(n_splits=10, test_size=0.20, random_state=101)
#cv = StratifiedShuffleSplit(n_splits=10,train_size=0.8)
print (cvx)

ShuffleSplit(n_splits=10, random_state=101, test_size=0.2, train_size=None)


In [0]:
features = ['isRe', 'underscore', 'priority', 'isInReplyTo',
            'sortedRec', 'subPunc', 'multipartText', 'hasImages', 'isPGPsigned',
            'subSpamWords', 'noHost', 'numEnd', 'isYelling', 'isOrigMsg', 'isDear',
            'isWrote', 'numLines', 'bodyCharCt', 'subExcCt', 'subQuesCt', 'numAtt',
            'numRec', 'perCaps', 'hour', 'perHTML', 'subBlanks', 'forwards',
            'avgWordLen', 'numDlr']

X = dfNoNa[features].copy()

#scaler = StandardScaler()
#scaler.fit(X2)

#This makes our model's coefficients take on the same scale for accurate feature importance analysis
#Notice we scaled the data before the cross validation
#X = scaler.transform(X2)

Y= dfNoNa[['isSpam']].copy()
y = Y.values

In [0]:

# Missing Data Summary
countOfNan = pd.Series(X.isnull().sum()) 
DataType = pd.Series(X.dtypes) 
  
frame = { 'datatype': DataType, 'count of Nan': countOfNan } 
result = pd.DataFrame(frame) 
print(result)


              datatype  count of Nan
isRe             int64             0
underscore       int64             0
priority         int64             0
isInReplyTo      int64             0
sortedRec        int64             0
subPunc          int64             0
multipartText    int64             0
hasImages        int64             0
isPGPsigned      int64             0
subSpamWords   float64             7
noHost         float64             0
numEnd           int64             0
isYelling      float64             7
isOrigMsg        int64             0
isDear           int64             0
isWrote          int64             0
numLines         int64             0
bodyCharCt       int64             0
subExcCt       float64            20
subQuesCt      float64            20
numAtt         float64             0
numRec         float64           282
perCaps        float64             0
hour           float64             0
perHTML        float64             0
subBlanks      float64            20
f

In [0]:
from sklearn.impute import SimpleImputer
imp=SimpleImputer(missing_values=np.NaN, strategy = 'mean')
idf=pd.DataFrame(imp.fit_transform(X))
idf.columns=X.columns
idf.index=X.index

### Q1: BUILD/EVALUATE TREE-BASED MODEL FOR PREDICTING "SPAM"

In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, TimeSeriesSplit, StratifiedShuffleSplit
from sklearn import metrics as mt
X1_train, X1_test, y1_train, y1_test = train_test_split(idf, y, test_size = 0.2, random_state = 101)

#impute missing values for noHost and numRec using the mean from the training set
X1_train.loc[X1_train['numRec'].isnull(),'numRec'] = X1_train.numRec.mean(skipna = True)
X1_test.loc[X1_test['numRec'].isnull(),'numRec'] = X1_train.numRec.mean(skipna = True)
X1_train.loc[X1_train['noHost'].isnull(),'noHost'] = X1_train.noHost.mean(skipna = True)
X1_test.loc[X1_test['noHost'].isnull(),'noHost'] = X1_train.noHost.mean(skipna = True)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_l

In [0]:
%%time
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA 
from sklearn.pipeline import Pipeline
import numpy as np



gnb_clf=GaussianNB()
gnb_clf.fit(X1_train,y1_train)
yhatgnb = gnb_clf.predict(X1_test)
print ('accuracy:', mt.accuracy_score(y1_test,yhatgnb))



accuracy: 0.9026737967914439
CPU times: user 9.49 ms, sys: 1.59 ms, total: 11.1 ms
Wall time: 14.1 ms


/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
for fold, (train, test) in enumerate(cvx.split(idf,y)):
    print ('Next Evaluation:')
    # train the decision tree algorithm
    gnb_clf.fit(idf.iloc[train],y[train])
    yhat = gnb_clf.predict(idf.iloc[test])
    print ('accuracy:', mt.accuracy_score(y[test],yhat),'\n')
    conf = mt.confusion_matrix(y[test],yhat)
    print("confusion matrix\n",conf,'\n')
    print("Precision Score is: {}" .format(precision_score(y[test],yhat, average='weighted')),'\n')
    print("Recall Score is: {}" .format(recall_score(y[test],yhat, average='weighted')),'\n')
    print("F1 Score is: {}" .format(f1_score(y[test],yhat, average='weighted')),'\n')

Next Evaluation:
accuracy: 0.9026737967914439 

confusion matrix
 [[1313   86]
 [  96  375]] 

Precision Score is: 0.902040547870718 

Recall Score is: 0.9026737967914439 

F1 Score is: 0.902324962152026 

Next Evaluation:
accuracy: 0.895187165775401 

confusion matrix
 [[1314   63]
 [ 133  360]] 

Precision Score is: 0.8930526574691318 

Recall Score is: 0.895187165775401 

F1 Score is: 0.8924813344924184 

Next Evaluation:
accuracy: 0.9042780748663102 

confusion matrix
 [[1296   74]
 [ 105  395]] 

Precision Score is: 0.9029048569386933 

Recall Score is: 0.9042780748663102 

F1 Score is: 0.9032823540042827 

Next Evaluation:
accuracy: 0.8946524064171123 

confusion matrix
 [[1293   78]
 [ 119  380]] 

Precision Score is: 0.8927662297781652 

Recall Score is: 0.8946524064171123 

F1 Score is: 0.8931717506253282 

Next Evaluation:
accuracy: 0.9112299465240642 

confusion matrix
 [[1303   65]
 [ 101  401]] 

Precision Score is: 0.909929563830094 

Recall Score is: 0.9112299465240642 


/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = col